In [14]:
import numpy as np
import pandas as pd
import random
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def hitung_akg(berat_badan, tinggi, usia, jenis_kelamin, faktor_aktivitas):
    if jenis_kelamin.lower() == 'pria':
        akg = 662 - (9.53 * usia) + faktor_aktivitas * ((15.91 * berat_badan) + (539.6 * tinggi / 100))
    elif jenis_kelamin.lower() == 'wanita':
        akg = 354 - (6.91 * usia) + faktor_aktivitas * ((9.36 * berat_badan) + (726 * tinggi / 100))
    else:
        return "Jenis kelamin tidak valid. Masukkan 'pria' atau 'wanita'."

    karbohidrat_kalori = 0.6 * akg
    protein_kalori = 0.15 * akg
    lemak_kalori = 0.25 * akg

    karbohidrat_gram = karbohidrat_kalori / 4
    protein_gram = protein_kalori / 4
    lemak_gram = lemak_kalori / 9

    return {
        "total_kalori": akg,
        "karbohidrat_gram": karbohidrat_gram,
        "protein_gram": protein_gram,
        "lemak_gram": lemak_gram
    }

berat_badan = 60 
tinggi = 176  
usia = 300 
jenis_kelamin = 'pria'  
faktor_aktivitas = 1.55 

kebutuhan_gizi = hitung_akg(berat_badan, tinggi, usia, jenis_kelamin, faktor_aktivitas)
print("Total Kalori:", kebutuhan_gizi["total_kalori"], "kalori per hari")
print("Kebutuhan Karbohidrat:", kebutuhan_gizi["karbohidrat_gram"], "gram per hari")
print("Kebutuhan Protein:", kebutuhan_gizi["protein_gram"], "gram per hari")
print("Kebutuhan Lemak:", kebutuhan_gizi["lemak_gram"], "gram per hari")

df = pd.read_csv('raw_clean.csv')

for col in ['Kalori', 'Karbohidrat', 'Protein', 'Lemak', 'Rating']:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

data = df[['Nama', 'Kalori', 'Karbohidrat', 'Protein', 'Lemak', 'Rating']].values

X = data[:, 1:-1].astype(float)  
y = data[:, -1].astype(float)   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

def calculate_nutrition_distance(features, target_features):
    return np.sqrt(np.sum((features - target_features) ** 2))

def fitness_function(combination, target_features, valid_X):
    combined_features = np.sum(valid_X[list(combination)], axis=0)
    distance = calculate_nutrition_distance(combined_features, target_features)
    return 1 / (distance + 1e-6)  

def recommend_meals_ga(target_features, population_size=100, num_generations=100):
    valid_indices = np.where(np.any(data[:, 1:-1].astype(float) != 0, axis=1))[0]
    valid_data = data[valid_indices]
    valid_X = valid_data[:, 1:-1].astype(float)
    num_meals = len(valid_X)

    population = [random.sample(range(num_meals), 3) for _ in range(population_size)]

    for generation in range(num_generations):
        fitness_scores = [fitness_function(individual, target_features, valid_X) for individual in population]

        best_individual = population[np.argmax(fitness_scores)]
        best_fitness = max(fitness_scores)

        if best_fitness > 0.99:
            break

        new_population = []
        for _ in range(population_size):
            if random.random() < 0.2: 
                new_individual = list(random.sample(range(num_meals), 3))
            else:  
                parent1, parent2 = random.sample(population, 2)
                crossover_point = random.randint(0, 2)
                new_individual = parent1[:crossover_point] + parent2[crossover_point:]
            new_population.append(new_individual)
        population = new_population

    best_combination = best_individual
    recommended_meals = valid_data[list(best_combination)]
    return recommended_meals

target_features = np.array([kebutuhan_gizi["total_kalori"], kebutuhan_gizi["karbohidrat_gram"], kebutuhan_gizi["protein_gram"], kebutuhan_gizi["lemak_gram"]])

recommended_meals = recommend_meals_ga(target_features)
print("Rekomendasi makanan dan kandungan gizinya:")
for meal in recommended_meals:
    print(f"{meal[0]} - Kalori: {meal[1]}, Karbohidrat: {meal[2]}, Protein: {meal[3]}, Lemak: {meal[4]}")


Total Kalori: 754.6588000000002 kalori per hari
Kebutuhan Karbohidrat: 113.19882000000003 gram per hari
Kebutuhan Protein: 28.299705000000007 gram per hari
Kebutuhan Lemak: 20.96274444444445 gram per hari
Mean Squared Error: 0.8546942041665556
Rekomendasi makanan dan kandungan gizinya:
Batatas kelapa umbi-umbian dibakar - Kalori: 110.0, Karbohidrat: 23.4, Protein: 1.4, Lemak: 1.1
Ayam Kalio  - Kalori: 203.0, Karbohidrat: 4.4, Protein: 16.0, Lemak: 13.5
Cumi keripik goreng - Kalori: 444.0, Karbohidrat: 71.3, Protein: 6.9, Lemak: 14.6
